In [1]:
import os, glob, logging
from typing import List, Optional

import pyspark.sql.functions as F
from pyspark.sql import DataFrame, SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import col, lit, when, datediff, avg, stddev, sum as _sum, max as _max

In [2]:
import pandas as pd
# Display all columns
pd.set_option('display.max_columns', None)

In [3]:
# Initialize SparkSession
spark = SparkSession.builder \
                    .appName("ExploreSilver") \
                    .master("local[8]") \
                    .config("spark.driver.memory", "8g") \
                    .config("spark.executor.memory", "8g") \
                    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/10 10:00:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
os.getcwd()

'/app/notebooks'

In [5]:
silver_dir = '/app/datamart/silver/'

### CLICKSTREAM

In [6]:
table_name = "feature_clickstream"
silver_click_directory = silver_dir + table_name + "/"
silver_click_directory

'/app/datamart/silver/feature_clickstream/'

In [7]:
parquet_files = glob.glob(os.path.join(silver_click_directory, "*.parquet"))
df_click = spark.read.parquet(*parquet_files)
df_click.show(5)

+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----------+-------------+
|fe_1|fe_2|fe_3|fe_4|fe_5|fe_6|fe_7|fe_8|fe_9|fe_10|fe_11|fe_12|fe_13|fe_14|fe_15|fe_16|fe_17|fe_18|fe_19|fe_20|Customer_ID|snapshot_date|
+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----------+-------------+
|  71| 243|  89| 149| 202|  56| 149| 207| 232|  111|  179|  340|  -17|  100|  115|   68|   39|   87|  -74|  240| CUS_0x1037|   2024-03-01|
| -60|  65|  30| -56|  25| 206|  68| 157| -36|   81|   77|   66|  169|   98|   75|  268|   95|   19|  151|  234| CUS_0x1069|   2024-03-01|
|  50|  76| 115| 114| -57| 136|  33|  69| 175|   92|  218|  151|   30|  204|  268|  181|  333|   70|  190|   23| CUS_0x114a|   2024-03-01|
|   1| 245|  72| 247| 212|  80| 144| 138|   7|  -86|  284|   35|  -46|  159|   81|   58|  340|   38|   66|  -36| CUS_0x1184|   2024-03-01|
|   8| 138| 164| 147| 184| 

In [8]:
df_click.agg(F.max("snapshot_date").alias("max_date")).collect()[0]["max_date"]

datetime.date(2024, 12, 1)

### ATTRIBUTES

In [9]:
table_name = "features_attributes"
silver_attr_directory = silver_dir + table_name + "/"
silver_attr_directory

'/app/datamart/silver/features_attributes/'

In [10]:
parquet_files = glob.glob(os.path.join(silver_attr_directory, "*.parquet"))
df_attr = spark.read.parquet(*parquet_files)
df_attr.show(5)

+-----------+-----------+---+-----------+------------+-------------+
|Customer_ID|       Name|Age|        SSN|  Occupation|snapshot_date|
+-----------+-----------+---+-----------+------------+-------------+
| CUS_0x10ac|      Zhouy| 29|780-50-4730|   Developer|   2024-08-01|
| CUS_0x10c5|      Moony| 24|041-74-6785|     Unknown|   2024-08-01|
| CUS_0x1145|Blenkinsopr| 24|426-31-9194|     Teacher|   2024-08-01|
| CUS_0x11ac|  Liana B.v| 26|835-92-7751|  Journalist|   2024-08-01|
| CUS_0x122c| Papadimasf| 48|883-73-9594|Entrepreneur|   2024-08-01|
+-----------+-----------+---+-----------+------------+-------------+
only showing top 5 rows



In [11]:
df_attr.agg(F.max("snapshot_date").alias("max_date")).collect()[0]["max_date"]

datetime.date(2024, 12, 1)

In [12]:
df_attr.select("Occupation").show()

+------------+
|  Occupation|
+------------+
|   Developer|
|     Unknown|
|     Teacher|
|  Journalist|
|Entrepreneur|
|   Scientist|
|      Doctor|
|   Developer|
|     Unknown|
|      Writer|
|    Mechanic|
|   Scientist|
|     Manager|
|     Manager|
|      Writer|
|      Doctor|
|    Engineer|
|   Architect|
|     Unknown|
|      Writer|
+------------+
only showing top 20 rows



In [13]:
df_attr = df_attr.withColumn(
    "is_occu_known",
    F.when(F.trim(F.col("Occupation")).rlike("(?i)^unknown$"), 0).otherwise(1)
)
df_attr.show(10)

+-----------+------------+---+-----------+------------+-------------+-------------+
|Customer_ID|        Name|Age|        SSN|  Occupation|snapshot_date|is_occu_known|
+-----------+------------+---+-----------+------------+-------------+-------------+
| CUS_0x10ac|       Zhouy| 29|780-50-4730|   Developer|   2024-08-01|            1|
| CUS_0x10c5|       Moony| 24|041-74-6785|     Unknown|   2024-08-01|            0|
| CUS_0x1145| Blenkinsopr| 24|426-31-9194|     Teacher|   2024-08-01|            1|
| CUS_0x11ac|   Liana B.v| 26|835-92-7751|  Journalist|   2024-08-01|            1|
| CUS_0x122c|  Papadimasf| 48|883-73-9594|Entrepreneur|   2024-08-01|            1|
| CUS_0x1274|      Whited|  0|298-61-0480|   Scientist|   2024-08-01|            1|
| CUS_0x1288|     Taylorq| 34|609-85-2725|      Doctor|   2024-08-01|            1|
| CUS_0x12cc|Lauren Tarat| 30|761-36-5969|   Developer|   2024-08-01|            1|
| CUS_0x1338|Olivia Orang| 40|250-62-9181|     Unknown|   2024-08-01|       

In [14]:
df_attr = df_attr.withColumn("age_band", when(col("Age") == 0, "Unknown") \
                                        .when(col("Age") < 25, "18-24") \
                                        .when((col("Age") >= 25) & (col("Age") < 35), "25-34") \
                                        .when((col("Age") >= 35) & (col("Age") < 45), "35-44") \
                                        .when((col("Age") >= 45) & (col("Age") < 55), "45-54") \
                                        .otherwise("55+"))
df_attr.show(10)

+-----------+------------+---+-----------+------------+-------------+-------------+--------+
|Customer_ID|        Name|Age|        SSN|  Occupation|snapshot_date|is_occu_known|age_band|
+-----------+------------+---+-----------+------------+-------------+-------------+--------+
| CUS_0x10ac|       Zhouy| 29|780-50-4730|   Developer|   2024-08-01|            1|   25-34|
| CUS_0x10c5|       Moony| 24|041-74-6785|     Unknown|   2024-08-01|            0|   18-24|
| CUS_0x1145| Blenkinsopr| 24|426-31-9194|     Teacher|   2024-08-01|            1|   18-24|
| CUS_0x11ac|   Liana B.v| 26|835-92-7751|  Journalist|   2024-08-01|            1|   25-34|
| CUS_0x122c|  Papadimasf| 48|883-73-9594|Entrepreneur|   2024-08-01|            1|   45-54|
| CUS_0x1274|      Whited|  0|298-61-0480|   Scientist|   2024-08-01|            1| Unknown|
| CUS_0x1288|     Taylorq| 34|609-85-2725|      Doctor|   2024-08-01|            1|   25-34|
| CUS_0x12cc|Lauren Tarat| 30|761-36-5969|   Developer|   2024-08-01| 

In [15]:
for band in ["Unknown", "18-24", "25-34", "35-44", "45-54", "55+"]:
            df_attr = df_attr.withColumn(f"age_band_{band.replace('-','_').replace('+','')}",
                                           when(col("age_band")==band, 1).otherwise(0))

In [16]:
df_attr.show(10)

+-----------+------------+---+-----------+------------+-------------+-------------+--------+----------------+--------------+--------------+--------------+--------------+-----------+
|Customer_ID|        Name|Age|        SSN|  Occupation|snapshot_date|is_occu_known|age_band|age_band_Unknown|age_band_18_24|age_band_25_34|age_band_35_44|age_band_45_54|age_band_55|
+-----------+------------+---+-----------+------------+-------------+-------------+--------+----------------+--------------+--------------+--------------+--------------+-----------+
| CUS_0x10ac|       Zhouy| 29|780-50-4730|   Developer|   2024-08-01|            1|   25-34|               0|             0|             1|             0|             0|          0|
| CUS_0x10c5|       Moony| 24|041-74-6785|     Unknown|   2024-08-01|            0|   18-24|               0|             1|             0|             0|             0|          0|
| CUS_0x1145| Blenkinsopr| 24|426-31-9194|     Teacher|   2024-08-01|            1|   18-2

In [17]:
df_attr = df_attr.drop("Age", "Name", "SSN", "Occupation","age_band")
df_attr.show(10)

+-----------+-------------+-------------+----------------+--------------+--------------+--------------+--------------+-----------+
|Customer_ID|snapshot_date|is_occu_known|age_band_Unknown|age_band_18_24|age_band_25_34|age_band_35_44|age_band_45_54|age_band_55|
+-----------+-------------+-------------+----------------+--------------+--------------+--------------+--------------+-----------+
| CUS_0x10ac|   2024-08-01|            1|               0|             0|             1|             0|             0|          0|
| CUS_0x10c5|   2024-08-01|            0|               0|             1|             0|             0|             0|          0|
| CUS_0x1145|   2024-08-01|            1|               0|             1|             0|             0|             0|          0|
| CUS_0x11ac|   2024-08-01|            1|               0|             0|             1|             0|             0|          0|
| CUS_0x122c|   2024-08-01|            1|               0|             0|          

### FINANCIALS

In [18]:
table_name = "features_financials"
silver_fin_directory = silver_dir + table_name + "/"
silver_fin_directory

'/app/datamart/silver/features_financials/'

In [19]:
parquet_files = glob.glob(os.path.join(silver_fin_directory, "*.parquet"))
df_fin = spark.read.parquet(*parquet_files)
df_fin.show(5)

+-----------+-------------+---------------------+-----------------+---------------+-------------+-----------+--------------------+-------------------+----------------------+--------------------+--------------------+----------+----------------+------------------------+---------------------+-------------------+-----------------------+--------------------+---------------+-------------+------------------+
|Customer_ID|Annual_Income|Monthly_Inhand_Salary|Num_Bank_Accounts|Num_Credit_Card|Interest_Rate|Num_of_Loan|        Type_of_Loan|Delay_from_due_date|Num_of_Delayed_Payment|Changed_Credit_Limit|Num_Credit_Inquiries|Credit_Mix|Outstanding_Debt|Credit_Utilization_Ratio|Payment_of_Min_Amount|Total_EMI_per_month|Amount_invested_monthly|   Payment_Behaviour|Monthly_Balance|snapshot_date|Credit_History_Age|
+-----------+-------------+---------------------+-----------------+---------------+-------------+-----------+--------------------+-------------------+----------------------+-----------------

In [20]:
df_fin.agg(F.max("snapshot_date").alias("max_date")).collect()[0]["max_date"]

datetime.date(2024, 12, 1)

In [21]:
valid_behaviours = [
        "Low_spent_Small_value_payments",
        "High_spent_Medium_value_payments",
        "Low_spent_Medium_value_payments",
        "High_spent_Large_value_payments",
        "High_spent_Small_value_payments",
        "Low_spent_Large_value_payments",
    ]
bad_tokens = ["na","n/a","none","null","-","?","unknown","undefined","nan"]
raw = F.trim(F.col("Payment_Behaviour"))
only_letters_underscores = F.regexp_replace(raw, r"[^A-Za-z_]", "")
df_fin = df_fin.withColumn(
    "Payment_Behaviour",
    F.when(
        raw.isNull()
        | (F.length(raw) == 0)
        | F.lower(raw).isin(bad_tokens)
        | (raw != only_letters_underscores)
        | (~raw.isin(valid_behaviours)),
        F.lit("Unknown"),
    ).otherwise(raw)
)
df_fin.show(5)

+-----------+-------------+---------------------+-----------------+---------------+-------------+-----------+--------------------+-------------------+----------------------+--------------------+--------------------+----------+----------------+------------------------+---------------------+-------------------+-----------------------+--------------------+---------------+-------------+------------------+
|Customer_ID|Annual_Income|Monthly_Inhand_Salary|Num_Bank_Accounts|Num_Credit_Card|Interest_Rate|Num_of_Loan|        Type_of_Loan|Delay_from_due_date|Num_of_Delayed_Payment|Changed_Credit_Limit|Num_Credit_Inquiries|Credit_Mix|Outstanding_Debt|Credit_Utilization_Ratio|Payment_of_Min_Amount|Total_EMI_per_month|Amount_invested_monthly|   Payment_Behaviour|Monthly_Balance|snapshot_date|Credit_History_Age|
+-----------+-------------+---------------------+-----------------+---------------+-------------+-----------+--------------------+-------------------+----------------------+-----------------

#### Debt to income

In [22]:
fin_cols_keep = [
            "Annual_Income","Outstanding_Debt","Payment_Behaviour","Credit_Mix",
            "Type_of_Loan","Credit_History_Age","Num_of_Loan","Num_of_Delayed_Payment",
            "Delay_from_due_date","snapshot_date"
        ]
fin_pre = df_fin.select("Customer_ID", *fin_cols_keep)
fin_pre.show(5)

+-----------+-------------+----------------+--------------------+----------+--------------------+------------------+-----------+----------------------+-------------------+-------------+
|Customer_ID|Annual_Income|Outstanding_Debt|   Payment_Behaviour|Credit_Mix|        Type_of_Loan|Credit_History_Age|Num_of_Loan|Num_of_Delayed_Payment|Delay_from_due_date|snapshot_date|
+-----------+-------------+----------------+--------------------+----------+--------------------+------------------+-----------+----------------------+-------------------+-------------+
| CUS_0x10ac|    16718.645|          853.41|             Unknown|         _|Credit-Builder Lo...|             195.0|          4|                    11|                 26|   2024-08-01|
| CUS_0x10c5|    158623.16|         1134.83|High_spent_Medium...|      Good|         Payday Loan|             362.0|          1|                     5|                 10|   2024-08-01|
| CUS_0x1145|     17975.32|          1263.1|Low_spent_Small_v...|     

In [23]:
# Capacity
features = fin_pre.withColumn("DTI",
                when(col("Annual_Income") > 0,
                     col("Outstanding_Debt") / col("Annual_Income")))
features = features.withColumn("log_Annual_Income",
                when(col("Annual_Income") > 0, F.log(col("Annual_Income"))))
features.show(5)

+-----------+-------------+----------------+--------------------+----------+--------------------+------------------+-----------+----------------------+-------------------+-------------+--------------------+------------------+
|Customer_ID|Annual_Income|Outstanding_Debt|   Payment_Behaviour|Credit_Mix|        Type_of_Loan|Credit_History_Age|Num_of_Loan|Num_of_Delayed_Payment|Delay_from_due_date|snapshot_date|                 DTI| log_Annual_Income|
+-----------+-------------+----------------+--------------------+----------+--------------------+------------------+-----------+----------------------+-------------------+-------------+--------------------+------------------+
| CUS_0x10ac|    16718.645|          853.41|             Unknown|         _|Credit-Builder Lo...|             195.0|          4|                    11|                 26|   2024-08-01|0.051045404521243475| 9.724279814647304|
| CUS_0x10c5|    158623.16|         1134.83|High_spent_Medium...|      Good|         Payday Loan

#### Behaviorals

In [24]:
pb_vals = [
            "High_spent_Small_value_payments", "Low_spent_Large_value_payments",
            "Low_spent_Medium_value_payments","Low_spent_Small_value_payments",
            "High_spent_Medium_value_payments","High_spent_Large_value_payments"
        ]
for v in pb_vals:
    features = features.withColumn(f"Payment_Behaviour_{v.replace(' ','_').replace('-','_')}",
                                   when(col("Payment_Behaviour")==v,1).otherwise(0))
for v in ["Standard","Good","Bad"]:
    features = features.withColumn(f"Credit_Mix_{v}", when(col("Credit_Mix")==v,1).otherwise(0))
loan_types = [
    "Auto Loan","Credit-Builder Loan","Personal Loan","Home Equity Loan",
    "Mortgage Loan","Student Loan","Debt Consolidation Loan","Payday Loan"
]
for v in loan_types:
    features = features.withColumn(f"Type_of_Loan_{v.replace(' ','_').replace('-','_')}",
                                   when(col("Type_of_Loan").contains(v),1).otherwise(0))

# ✅ DROP ORIGINAL CATEGORICAL COLUMNS
features = features.drop("Payment_Behaviour", "Credit_Mix", "Type_of_Loan")

In [25]:
features.show()

+-----------+-------------+----------------+------------------+-----------+----------------------+-------------------+-------------+--------------------+------------------+-------------------------------------------------+------------------------------------------------+-------------------------------------------------+------------------------------------------------+--------------------------------------------------+-------------------------------------------------+-------------------+---------------+--------------+----------------------+--------------------------------+--------------------------+-----------------------------+--------------------------+-------------------------+------------------------------------+------------------------+
|Customer_ID|Annual_Income|Outstanding_Debt|Credit_History_Age|Num_of_Loan|Num_of_Delayed_Payment|Delay_from_due_date|snapshot_date|                 DTI| log_Annual_Income|Payment_Behaviour_High_spent_Small_value_payments|Payment_Behaviour_Low_spent

### APPLICATION

In [26]:
app_store = '/app/datamart/gold/application_store'

In [27]:
df_app = spark.read.parquet(app_store)
df_app.show(5)

+-----------+--------+------+----------------+
|Customer_ID|loan_amt|tenure|application_date|
+-----------+--------+------+----------------+
| CUS_0x416d|   10000|    10|      2024-08-01|
| CUS_0x20fa|   10000|    10|      2024-08-01|
| CUS_0x43fc|   10000|    10|      2024-08-01|
| CUS_0x4c64|   10000|    10|      2024-08-01|
| CUS_0x1572|   10000|    10|      2024-08-01|
+-----------+--------+------+----------------+
only showing top 5 rows



In [28]:
df_app.agg(F.min("application_date").alias("min_date")).collect()[0]["min_date"]

datetime.date(2023, 1, 1)

In [29]:
df_app.agg(F.max("application_date").alias("max_date")).collect()[0]["max_date"]

datetime.date(2025, 1, 1)

In [30]:
df_app = df_app.drop("loan_amt","tenure")
df_app.show(5)

+-----------+----------------+
|Customer_ID|application_date|
+-----------+----------------+
| CUS_0x416d|      2024-08-01|
| CUS_0x20fa|      2024-08-01|
| CUS_0x43fc|      2024-08-01|
| CUS_0x4c64|      2024-08-01|
| CUS_0x1572|      2024-08-01|
+-----------+----------------+
only showing top 5 rows



### JOIN EXPLORATION

#### CLICKSTREAMS

In [31]:
df_click_filtered = df_click.join(df_app, on='Customer_ID', how='right') 
df_click_filtered.show(10)

+-----------+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-------------+----------------+
|Customer_ID|fe_1|fe_2|fe_3|fe_4|fe_5|fe_6|fe_7|fe_8|fe_9|fe_10|fe_11|fe_12|fe_13|fe_14|fe_15|fe_16|fe_17|fe_18|fe_19|fe_20|snapshot_date|application_date|
+-----------+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-------------+----------------+
| CUS_0x416d|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL| NULL| NULL| NULL| NULL| NULL| NULL| NULL| NULL| NULL| NULL| NULL|         NULL|      2024-08-01|
| CUS_0x20fa|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL| NULL| NULL| NULL| NULL| NULL| NULL| NULL| NULL| NULL| NULL| NULL|         NULL|      2024-08-01|
| CUS_0x43fc|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL| NULL| NULL| NULL| NULL| NULL| NULL| NULL| NULL| NULL| NULL| NULL|         NULL|      2024-08-01|
| CUS_0x4c64|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL| NULL|

In [32]:
df_click_filtered.agg(F.max("application_date").alias("max_date")).collect()[0]["max_date"]

datetime.date(2025, 1, 1)

In [33]:
df_click_filtered = df_click_filtered.filter(F.col("snapshot_date") <= F.col("application_date"))        
df_click_filtered.show(10)

+-----------+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-------------+----------------+
|Customer_ID|fe_1|fe_2|fe_3|fe_4|fe_5|fe_6|fe_7|fe_8|fe_9|fe_10|fe_11|fe_12|fe_13|fe_14|fe_15|fe_16|fe_17|fe_18|fe_19|fe_20|snapshot_date|application_date|
+-----------+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-------------+----------------+
| CUS_0x102e| -40|  86| 157| 226|  51|  88| 334|  24| 248|  161|  -43|   -8|  208|  171|  -43|  215|  237|  165|    5|  163|   2024-03-01|      2024-04-01|
| CUS_0x109d|  35| -80| 170|-212| 116|  62|  20| 283| 203|   54|  321| -152|  173|  220|  107|   -8|   -7|  375|  154|  213|   2024-03-01|      2024-04-01|
| CUS_0x112e| 221|-181| 214| -75|  93| 117| 110| 239| 154|  217|   98|   95|  156|  104|   42|  135|  -43|  105|   88|  145|   2024-03-01|      2024-04-01|
| CUS_0x1183| 122|  10| 115| 162| -23| 365|  34| -35| 111|  225|

In [34]:
df_click.filter(F.col("snapshot_date") == "2024-10-01").show()

+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----------+-------------+
|fe_1|fe_2|fe_3|fe_4|fe_5|fe_6|fe_7|fe_8|fe_9|fe_10|fe_11|fe_12|fe_13|fe_14|fe_15|fe_16|fe_17|fe_18|fe_19|fe_20|Customer_ID|snapshot_date|
+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----------+-------------+
| 199| 157| -33|  43| 119| 391|   4| 124| 222|  317|   70|   55|  107|  115|  151|   11|    6|  -70| -110|   86| CUS_0x1037|   2024-10-01|
|-185|   7| 233|  54|  84| 183|  24|  47|  72|  157|  285|  205|  248|  131|   82|   92|  -65|  -72|  202|  239| CUS_0x1069|   2024-10-01|
|  45| 246| 222|-171| 147| -21|  62| -24| 149|  105|  197|  144|   96|  -26|  291|  204|   34|   76|   60|   -5| CUS_0x114a|   2024-10-01|
|  73|  11| 128| -65| 113|-103| 113| 100| 181|  -91|  223|  -13|   26|  195|   46|   38|   25|  186|  192|   45| CUS_0x1184|   2024-10-01|
| 167|  38| 156| 158| 142| 

In [35]:
df_app.filter(col("Customer_ID") == "CUS_0x1297").show()

+-----------+----------------+
|Customer_ID|application_date|
+-----------+----------------+
| CUS_0x1297|      2023-01-01|
+-----------+----------------+



In [36]:
feature_cols = [f"fe_{i}" for i in range(1, 21)]
agg_exprs = [F.sum(F.col(c)).alias(f"{c}_sum_all") for c in feature_cols]
df_click_sum = (
    df_click_filtered.groupBy("Customer_ID", "application_date")
                     .agg(*agg_exprs)
)
df_click_sum.show(10)

[Stage 46:===========================================>              (6 + 2) / 8]

+-----------+----------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+
|Customer_ID|application_date|fe_1_sum_all|fe_2_sum_all|fe_3_sum_all|fe_4_sum_all|fe_5_sum_all|fe_6_sum_all|fe_7_sum_all|fe_8_sum_all|fe_9_sum_all|fe_10_sum_all|fe_11_sum_all|fe_12_sum_all|fe_13_sum_all|fe_14_sum_all|fe_15_sum_all|fe_16_sum_all|fe_17_sum_all|fe_18_sum_all|fe_19_sum_all|fe_20_sum_all|
+-----------+----------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+
| CUS_0x5978|      2024-04-01|        1341|        1650|        1523|        1711|        2210

In [37]:
df_click_ondate = df_click_filtered.filter(F.col("snapshot_date") == F.col("application_date"))
df_click_ondate = df_click_ondate.drop("application_date")
df_click_ondate.show(10)

+-----------+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-------------+
|Customer_ID|fe_1|fe_2|fe_3|fe_4|fe_5|fe_6|fe_7|fe_8|fe_9|fe_10|fe_11|fe_12|fe_13|fe_14|fe_15|fe_16|fe_17|fe_18|fe_19|fe_20|snapshot_date|
+-----------+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-------------+
| CUS_0x100b| 210| 129| 107| 118| -51|  67|  40|  88| 189|  123|  125|    2|   60|  242|  177|   29|  -94|  -38|  380| -199|   2024-03-01|
| CUS_0x1096| -60|  21| 214|  76|  32| -83| 179| 112| 106|   25|  140| -142|   84|   62|  165| -121|  149|  188|   28|   55|   2024-03-01|
| CUS_0x111c|  84| -41| 137| 328|  45|  93| -39| 248| 114|  212|  205|  239|   42| -181|   74|  -34|  -16|  221|   92|   62|   2024-03-01|
| CUS_0x112d| 195| 112| 262| 374|  98| 148| 136| 273|  33|  313|  -12|   55|  171|  290|   85|  -72|  -43|  122|  165|  -18|   2024-03-01|
| CUS_0x1204| 129| 173| 196

In [38]:
df_click_features = df_click_sum.join(df_click_ondate, on="Customer_ID", how='left')
df_click_features.show(10)

[Stage 54:===========>      (5 + 3) / 8][Stage 56:>                 (0 + 4) / 4]

+-----------+----------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-------------+
|Customer_ID|application_date|fe_1_sum_all|fe_2_sum_all|fe_3_sum_all|fe_4_sum_all|fe_5_sum_all|fe_6_sum_all|fe_7_sum_all|fe_8_sum_all|fe_9_sum_all|fe_10_sum_all|fe_11_sum_all|fe_12_sum_all|fe_13_sum_all|fe_14_sum_all|fe_15_sum_all|fe_16_sum_all|fe_17_sum_all|fe_18_sum_all|fe_19_sum_all|fe_20_sum_all|fe_1|fe_2|fe_3|fe_4|fe_5|fe_6|fe_7|fe_8|fe_9|fe_10|fe_11|fe_12|fe_13|fe_14|fe_15|fe_16|fe_17|fe_18|fe_19|fe_20|snapshot_date|
+-----------+----------------+------------+------------+------------+------------+------------+------------+------------+------------+------------

#### ALL FEATURES

In [39]:
df_features = df_click_features.join(features, how="left", on=["Customer_ID"]) \
                               .join(df_attr, how="left", on=["Customer_ID"])
df_features.show()

+-----------+----------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-------------+-------------+----------------+------------------+-----------+----------------------+-------------------+-------------+--------------------+------------------+-------------------------------------------------+------------------------------------------------+-------------------------------------------------+------------------------------------------------+--------------------------------------------------+-------------------------------------------------+-------------------+---------------+--------------+----------------------+--------------------------------+---------

In [40]:
numeric_cols = [f.name for f in df_features.schema.fields 
                if f.dataType.typeName() in ['integer', 'long', 'double', 'float']]

fill_dict = {col: 0 for col in numeric_cols}
df_features = df_features.fillna(fill_dict)
df_features.show()

+-----------+----------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-------------+-------------+----------------+------------------+-----------+----------------------+-------------------+-------------+--------------------+------------------+-------------------------------------------------+------------------------------------------------+-------------------------------------------------+------------------------------------------------+--------------------------------------------------+-------------------------------------------------+-------------------+---------------+--------------+----------------------+--------------------------------+---------

In [44]:
features.columns

['Customer_ID',
 'Annual_Income',
 'Outstanding_Debt',
 'Credit_History_Age',
 'Num_of_Loan',
 'Num_of_Delayed_Payment',
 'Delay_from_due_date',
 'snapshot_date',
 'DTI',
 'log_Annual_Income',
 'Payment_Behaviour_High_spent_Small_value_payments',
 'Payment_Behaviour_Low_spent_Large_value_payments',
 'Payment_Behaviour_Low_spent_Medium_value_payments',
 'Payment_Behaviour_Low_spent_Small_value_payments',
 'Payment_Behaviour_High_spent_Medium_value_payments',
 'Payment_Behaviour_High_spent_Large_value_payments',
 'Credit_Mix_Standard',
 'Credit_Mix_Good',
 'Credit_Mix_Bad',
 'Type_of_Loan_Auto_Loan',
 'Type_of_Loan_Credit_Builder_Loan',
 'Type_of_Loan_Personal_Loan',
 'Type_of_Loan_Home_Equity_Loan',
 'Type_of_Loan_Mortgage_Loan',
 'Type_of_Loan_Student_Loan',
 'Type_of_Loan_Debt_Consolidation_Loan',
 'Type_of_Loan_Payday_Loan']

In [45]:
df_attr.columns

['Customer_ID',
 'snapshot_date',
 'is_occu_known',
 'age_band_Unknown',
 'age_band_18_24',
 'age_band_25_34',
 'age_band_35_44',
 'age_band_45_54',
 'age_band_55']

In [46]:
df_click_features.columns

['Customer_ID',
 'application_date',
 'fe_1_sum_all',
 'fe_2_sum_all',
 'fe_3_sum_all',
 'fe_4_sum_all',
 'fe_5_sum_all',
 'fe_6_sum_all',
 'fe_7_sum_all',
 'fe_8_sum_all',
 'fe_9_sum_all',
 'fe_10_sum_all',
 'fe_11_sum_all',
 'fe_12_sum_all',
 'fe_13_sum_all',
 'fe_14_sum_all',
 'fe_15_sum_all',
 'fe_16_sum_all',
 'fe_17_sum_all',
 'fe_18_sum_all',
 'fe_19_sum_all',
 'fe_20_sum_all',
 'fe_1',
 'fe_2',
 'fe_3',
 'fe_4',
 'fe_5',
 'fe_6',
 'fe_7',
 'fe_8',
 'fe_9',
 'fe_10',
 'fe_11',
 'fe_12',
 'fe_13',
 'fe_14',
 'fe_15',
 'fe_16',
 'fe_17',
 'fe_18',
 'fe_19',
 'fe_20',
 'snapshot_date']

In [52]:
features = features.drop("snapshot_date")
df_attr = df_attr.drop("snapshot_date")
df_click_features = df_click_features.drop("snapshot_date", "application_date")
df_features = df_app.join(features, how="left", on=["Customer_ID"]) \
                    .join(df_attr, how="left", on=["Customer_ID"]) \
                    .join(df_click_features, how="left", on=["Customer_ID"])
df_features.show(5)

+-----------+----------------+-------------+----------------+------------------+-----------+----------------------+-------------------+--------------------+------------------+-------------------------------------------------+------------------------------------------------+-------------------------------------------------+------------------------------------------------+--------------------------------------------------+-------------------------------------------------+-------------------+---------------+--------------+----------------------+--------------------------------+--------------------------+-----------------------------+--------------------------+-------------------------+------------------------------------+------------------------+-------------+----------------+--------------+--------------+--------------+--------------+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+-------------+-------------

In [53]:
numeric_cols = [f.name for f in df_features.schema.fields 
                if f.dataType.typeName() in ['integer', 'long', 'double', 'float']]

fill_dict = {col: 0 for col in numeric_cols}
df_features = df_features.fillna(fill_dict)
df_features.show()

+-----------+----------------+-------------+----------------+------------------+-----------+----------------------+-------------------+--------------------+------------------+-------------------------------------------------+------------------------------------------------+-------------------------------------------------+------------------------------------------------+--------------------------------------------------+-------------------------------------------------+-------------------+---------------+--------------+----------------------+--------------------------------+--------------------------+-----------------------------+--------------------------+-------------------------+------------------------------------+------------------------+-------------+----------------+--------------+--------------+--------------+--------------+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+-------------+-------------

In [54]:
check_null = df_features.filter(col("Customer_ID") == "CUS_0x416d").toPandas()
check_null.head()

,Customer_ID,application_date,Annual_Income,Outstanding_Debt,Credit_History_Age,Num_of_Loan,Num_of_Delayed_Payment,Delay_from_due_date,DTI,log_Annual_Income,Payment_Behaviour_High_spent_Small_value_payments,Payment_Behaviour_Low_spent_Large_value_payments,Payment_Behaviour_Low_spent_Medium_value_payments,Payment_Behaviour_Low_spent_Small_value_payments,Payment_Behaviour_High_spent_Medium_value_payments,Payment_Behaviour_High_spent_Large_value_payments,Credit_Mix_Standard,Credit_Mix_Good,Credit_Mix_Bad,Type_of_Loan_Auto_Loan,Type_of_Loan_Credit_Builder_Loan,Type_of_Loan_Personal_Loan,Type_of_Loan_Home_Equity_Loan,Type_of_Loan_Mortgage_Loan,Type_of_Loan_Student_Loan,Type_of_Loan_Debt_Consolidation_Loan,Type_of_Loan_Payday_Loan,is_occu_known,age_band_Unknown,age_band_18_24,age_band_25_34,age_band_35_44,age_band_45_54,age_band_55,fe_1_sum_all,fe_2_sum_all,fe_3_sum_all,fe_4_sum_all,fe_5_sum_all,fe_6_sum_all,fe_7_sum_all,fe_8_sum_all,fe_9_sum_all,fe_10_sum_all,fe_11_sum_all,fe_12_sum_all,fe_13_sum_all,fe_14_sum_all,fe_15_sum_all,fe_16_sum_all,fe_17_sum_all,fe_18_sum_all,fe_19_sum_all,fe_20_sum_all,fe_1,fe_2,fe_3,fe_4,fe_5,fe_6,fe_7,fe_8,fe_9,fe_10,fe_11,fe_12,fe_13,fe_14,fe_15,fe_16,fe_17,fe_18,fe_19,fe_20
0,CUS_0x416d,2024-08-01,44158.03125,1298.22998,191.0,1,0,13,0.0294,10.69553,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### WRITE TO PARQUET

In [42]:
gold_feature_store_directory = "/app/datamart/gold/feature_store/"
silver_attr_directory = "/app/datamart/silver/features_attributes/"
silver_fin_directory = "/app/datamart/silver/features_financials/"
silver_clickstream_directory = "/app/datamart/silver/feature_clickstream/"

In [43]:
# --- Write by snapshot_date ---
df_features.write.mode("overwrite") \
           .partitionBy("snapshot_date") \
           .parquet(gold_feature_store_directory)
print("Written to gold/datamart/feature_store")

AnalysisException: [AMBIGUOUS_REFERENCE] Reference `snapshot_date` is ambiguous, could be: [`snapshot_date`, `snapshot_date`, `snapshot_date`].

In [ ]:
spark.stop()